In [39]:
!pip install mrjob

In [16]:
from json import load
import pandas as pd

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
PATH ='/content/drive/MyDrive/Big Data/Proyecto/dataset'

In [61]:
with open(PATH + '/MMHS150K_GT.json', 'r') as fp:
  data = load(fp)

In [19]:
df = pd.DataFrame.from_dict(data, orient='index')
df.head()

,img_url,labels,tweet_url,tweet_text,labels_str
1114679353714016256,http://pbs.twimg.com/tweet_video_thumb/D3gi9MH...,"[4, 1, 3]",https://twitter.com/user/status/11146793537140...,@FriskDontMiss Nigga https://t.co/cAsaLWEpue,"[Religion, Racist, Homophobe]"
1063020048816660480,http://pbs.twimg.com/ext_tw_video_thumb/106301...,"[5, 5, 5]",https://twitter.com/user/status/10630200488166...,My horses are retarded https://t.co/HYhqc6d5WN,"[OtherHate, OtherHate, OtherHate]"
1108927368075374593,http://pbs.twimg.com/media/D2OzhzHUwAADQjd.jpg,"[0, 0, 0]",https://twitter.com/user/status/11089273680753...,“NIGGA ON MA MOMMA YOUNGBOY BE SPITTING REAL S...,"[NotHate, NotHate, NotHate]"
1114558534635618305,http://pbs.twimg.com/ext_tw_video_thumb/111401...,"[1, 0, 0]",https://twitter.com/user/status/11145585346356...,RT xxSuGVNGxx: I ran into this HOLY NIGGA TODA...,"[Racist, NotHate, NotHate]"
1035252480215592966,http://pbs.twimg.com/media/Dl30pGIU8AAVGxO.jpg,"[1, 0, 1]",https://twitter.com/user/status/10352524802155...,“EVERYbody calling you Nigger now!” https://t....,"[Racist, NotHate, Racist]"


In [66]:
df.to_csv("data.csv", index=False, sep='∑')

In [67]:
!head data.csv

img_url∑labels∑tweet_url∑tweet_text∑labels_str
http://pbs.twimg.com/tweet_video_thumb/D3gi9MHWAAAgfl7.jpg∑[4, 1, 3]∑https://twitter.com/user/status/1114679353714016256∑@FriskDontMiss Nigga https://t.co/cAsaLWEpue∑['Religion', 'Racist', 'Homophobe']
http://pbs.twimg.com/ext_tw_video_thumb/1063019643709747200/pu/img/wK5HgoX6tFfxWJmi.jpg∑[5, 5, 5]∑https://twitter.com/user/status/1063020048816660480∑My horses are retarded https://t.co/HYhqc6d5WN∑['OtherHate', 'OtherHate', 'OtherHate']
http://pbs.twimg.com/media/D2OzhzHUwAADQjd.jpg∑[0, 0, 0]∑https://twitter.com/user/status/1108927368075374593∑“NIGGA ON MA MOMMA YOUNGBOY BE SPITTING REAL SHIT NIGGA” https://t.co/UczofqHrLq∑['NotHate', 'NotHate', 'NotHate']
http://pbs.twimg.com/ext_tw_video_thumb/1114018095348084738/pu/img/m6oP5fBm2RCcq5Wn.jpg∑[1, 0, 0]∑https://twitter.com/user/status/1114558534635618305∑RT xxSuGVNGxx: I ran into this HOLY NIGGA TODAY 😭😭😭😭 https://t.co/Wa6Spl9kIw∑['Racist', 'NotHate', 'NotHate']
http://pbs.twimg.com/media/Dl3

In [31]:
!ls

clean_data.py  data.csv  drive	sample_data


In [79]:
%%file clean_data.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

class MRDataCleansing(MRJob):
  def __init__(self, *args, **kwargs):
    super(MRDataCleansing, self).__init__(*args, **kwargs)
    self._stopwords = set(stopwords.words("english") + ['\'', '’'])
    self._sep = '∑'

  def steps(self):
    return [
            MRStep(mapper=self.mapper, reducer=self.reducer)
    ]

  def limpiar(self, t):
    without_links = ' '.join([word for word in t.split(' ') if word[:4] != 'http']) # quitar los links
    tokens = word_tokenize(without_links) # tokenizar (dividir en tokens)
    tokens = map(lambda token: token.encode('ascii', 'ignore').decode('ascii'), tokens) # quitar los emojis
    tokens_without_sw = [word.lower() for word in tokens if word.isalpha() and not word in self._stopwords] # filtrar puntuacion y poner todo a minuscula
    return ' '.join(tokens_without_sw)

  def mapper(self, _, line):
    if line == f'img_url{self._sep}labels{self._sep}tweet_url{self._sep}tweet_text{self._sep}labels_str':
      yield None, line
    else:
      row_values = line.split(self._sep)
      row_values[3] = self.limpiar(row_values[3])
      yield None, self._sep.join(row_values)

  def reducer(self, key, values):
    for value in values:
        yield None, value

if __name__ == '__main__':
  MRDataCleansing.run()

Overwriting clean_data.py


In [93]:
!python3 clean_data.py -r local *.csv --output-dir=clean_data_out --no-output

No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/clean_data.root.20211128.222526.204309
Running step 1 of 1...
tcmalloc: large alloc 2150473728 bytes == 0x5581c9986000 @  0x7fa8200f91e7 0x5581c8e8d718 0x5581c8e8c5a1 0x7fa81fad7bf7 0x5581c8e8d02a
job output is in clean_data_out
Removing temp directory /tmp/clean_data.root.20211128.222526.204309...


In [81]:
!ls

clean_data_out	clean_data.py  data.csv  drive	sample_data


In [82]:
!head -n 1 data.csv

img_url∑labels∑tweet_url∑tweet_text∑labels_str


## Nota

Como puede ver profesor, MRJob genera un archivo de la forma:

`"key1"    "value1"`

`"key2"    "value2"`

`"key3"    "value3"`

`...`

Lo cual invalida el csv ya que se generaría un csv válido si MRJob genera el archivo de la forma:

`value1`

`value2`

`value3`

`...`

Se generaría un csv válido, el cual ya tiene el texto de los tweets limpios.

In [94]:
!head -n 10 ./clean_data_out/part-00000

null	"img_url\u2211labels\u2211tweet_url\u2211tweet_text\u2211labels_str"
null	"http://pbs.twimg.com/tweet_video_thumb/D3gi9MHWAAAgfl7.jpg\u2211[4, 1, 3]\u2211https://twitter.com/user/status/1114679353714016256\u2211friskdontmiss nigga\u2211['Religion', 'Racist', 'Homophobe']"
null	"http://pbs.twimg.com/ext_tw_video_thumb/1063019643709747200/pu/img/wK5HgoX6tFfxWJmi.jpg\u2211[5, 5, 5]\u2211https://twitter.com/user/status/1063020048816660480\u2211my horses retarded\u2211['OtherHate', 'OtherHate', 'OtherHate']"
null	"http://pbs.twimg.com/media/D2OzhzHUwAADQjd.jpg\u2211[0, 0, 0]\u2211https://twitter.com/user/status/1108927368075374593\u2211nigga on ma momma youngboy be spitting real shit nigga\u2211['NotHate', 'NotHate', 'NotHate']"
null	"http://pbs.twimg.com/ext_tw_video_thumb/1114018095348084738/pu/img/m6oP5fBm2RCcq5Wn.jpg\u2211[1, 0, 0]\u2211https://twitter.com/user/status/1114558534635618305\u2211rt xxsugvngxx i ran holy nigga today\u2211['Racist', 'NotHate', 'NotHate']"
null	"http://p

In [ ]:
df = pd.read_csv('clean_data_out/part-00000', sep='∑')
df.head()